In [1]:
import pandas as pd
fname = "/home/devmiftahul/trading_model/from_finrl-tutorials_git/processed_by_gavin/preprocessed_data.csv"
df = pd.read_csv(fname)

In [2]:
df

,date,tick,close,high,low,open,volume
0,1993-01-04,AALI,1.0,1.0,1.0,1.0,1.0
1,1993-01-04,ABBA,1.0,1.0,1.0,1.0,1.0
2,1993-01-04,ABDA,950.0,950.0,950.0,950.0,1.0
3,1993-01-04,ABMM,1.0,1.0,1.0,1.0,1.0
4,1993-01-04,ACAP,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...
8523195,2024-03-08,ZATA,50.0,50.0,50.0,50.0,200.0
8523196,2024-03-08,ZBRA,462.0,462.0,444.0,448.0,724100.0
8523197,2024-03-08,ZINC,50.0,50.0,50.0,50.0,1.0
8523198,2024-03-08,ZONE,1125.0,1130.0,1120.0,1120.0,113400.0


In [4]:
tickers = df["tick"].unique()
len(tickers)

1120

In [14]:
import pandas as pd
all_df = df.copy()
# Assuming the DataFrame is already loaded as 'df'
# nan_volume_count = all_df[all_df['volume'].isna()].groupby('tick').size()
nan_volume_count = all_df[all_df['volume'] == 1].groupby('tick').size()

# print("Number of rows with NaN volume for each ticker:")
print(nan_volume_count)

# Optional: If you want to see tickers with NaN volumes sorted from most to least
print("\nSorted from most NaN volumes to least:")
tickers = sorted(nan_volume_count.sort_values(ascending=True)[:30].keys().tolist())
# print(tickers[:10])
all_df = all_df[all_df["tick"].isin(tickers)]
print(all_df)

tick
AALI    1233
ABBA    4099
ABDA    5231
ABMM    5456
ACAP    7610
        ... 
ZATA    7290
ZBRA    3451
ZINC    6110
ZONE    6335
ZYRX    6891
Length: 1120, dtype: int64

Sorted from most NaN volumes to least:
               date  tick     close      high       low      open     volume
13       1993-01-04  ADMG      1.00      1.00      1.00      1.00        1.0
77       1993-01-04  ASII    108.20    110.00    108.20    109.40  5523860.0
130      1993-01-04  BDMN  30350.00  30350.00  30350.00  30350.00        1.0
176      1993-01-04  BNGA   6070.00   6070.00   6070.00   6070.00        1.0
177      1993-01-04  BNII   2537.27   2537.27   2537.27   2537.27        1.0
...             ...   ...       ...       ...       ...       ...        ...
8522970  2024-03-08  SMGR   5900.00   5950.00   5900.00   5950.00  6695300.0
8523047  2024-03-08  TKIM   6850.00   6950.00   6800.00   6850.00  1895200.0
8523082  2024-03-08  TSPC   1940.00   1960.00   1940.00   1955.00   839100.0
8523097  2024-0

In [15]:
dates = sorted(all_df["date"].unique())
date2day = {date:day for day, date in enumerate(dates, start=1)}
all_df["day"] = all_df["date"].map(date2day)

In [16]:
all_df

,date,tick,close,high,low,open,volume,day
13,1993-01-04,ADMG,1.00,1.00,1.00,1.00,1.0,1
77,1993-01-04,ASII,108.20,110.00,108.20,109.40,5523860.0,1
130,1993-01-04,BDMN,30350.00,30350.00,30350.00,30350.00,1.0,1
176,1993-01-04,BNGA,6070.00,6070.00,6070.00,6070.00,1.0,1
177,1993-01-04,BNII,2537.27,2537.27,2537.27,2537.27,1.0,1
...,...,...,...,...,...,...,...,...
8522970,2024-03-08,SMGR,5900.00,5950.00,5900.00,5950.00,6695300.0,7610
8523047,2024-03-08,TKIM,6850.00,6950.00,6800.00,6850.00,1895200.0,7610
8523082,2024-03-08,TSPC,1940.00,1960.00,1940.00,1955.00,839100.0,7610
8523097,2024-03-08,UNSP,110.00,112.00,105.00,110.00,83600.0,7610


In [18]:
def validate_tic_dates(all_df):
    """
    Validates that each ticker (tic) in the DataFrame has the same unique dates.
    
    Steps:
    1. Counts the unique dates for each ticker and saves it in a dictionary.
    2. Asserts that all tickers have the same unique dates and the same total unique dates.

    Parameters:
    - all_df (pd.DataFrame): The DataFrame containing stock data with 'date' and 'tic' columns.

    Returns:
    - dict: A dictionary with tickers as keys and their total unique dates as values.
    - int: Total number of unique dates across the dataset.

    Raises:
    - AssertionError: If any ticker has a different set or total count of unique dates.
    """
    # Group the data by 'tic' and count unique dates for each tic
    unique_dates_per_tic = all_df.groupby("tick")["date"].nunique().to_dict()

    # Check that all tickers have the same total unique dates
    total_unique_dates = len(all_df["date"].unique())
    assert all(val == total_unique_dates for val in unique_dates_per_tic.values()), \
        "Not all tickers have the same number of unique dates!"

    # Check that all tickers have the same set of dates
    unique_date_sets = all_df.groupby("tick")["date"].apply(lambda x: set(x)).to_dict()

    # Ensure that all tickers have the same set of dates
    date_sets = list(unique_date_sets.values())
    assert all(date_set == date_sets[0] for date_set in date_sets), \
        "Not all tickers share the same set of dates!"

    # Return the results
    return unique_dates_per_tic, total_unique_dates

# Example usage:
new_total_for_each_tic, total_unique_dates = validate_tic_dates(all_df)
print("Unique dates per ticker:", new_total_for_each_tic)
print("Total unique dates in the dataset:", total_unique_dates)

Unique dates per ticker: {'ADMG': 7610, 'ASII': 7610, 'BDMN': 7610, 'BNGA': 7610, 'BNII': 7610, 'BNLI': 7610, 'BRPT': 7610, 'CMNP': 7610, 'CPIN': 7610, 'CTRA': 7610, 'DILD': 7610, 'GGRM': 7610, 'GJTL': 7610, 'HMSP': 7610, 'INCO': 7610, 'INDF': 7610, 'INKP': 7610, 'INTP': 7610, 'ISAT': 7610, 'KLBF': 7610, 'MLPL': 7610, 'MPPA': 7610, 'MYOR': 7610, 'PNBN': 7610, 'SMCB': 7610, 'SMGR': 7610, 'TKIM': 7610, 'TSPC': 7610, 'UNSP': 7610, 'UNTR': 7610}
Total unique dates in the dataset: 7610


In [22]:
all_df = all_df.rename(columns={"tick": "tic"})
tickers = all_df["tic"].unique()
len(tickers)

30

In [25]:
import numpy as np
from finrl.config import INDICATORS
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(all_df)
processed = processed.copy()
processed = processed.fillna(0)
processed = processed.replace(np.inf,0)

BEFORE CLEAN:                date   tic     close      high       low      open     volume  \
13       1993-01-04  ADMG      1.00      1.00      1.00      1.00        1.0   
77       1993-01-04  ASII    108.20    110.00    108.20    109.40  5523860.0   
130      1993-01-04  BDMN  30350.00  30350.00  30350.00  30350.00        1.0   
176      1993-01-04  BNGA   6070.00   6070.00   6070.00   6070.00        1.0   
177      1993-01-04  BNII   2537.27   2537.27   2537.27   2537.27        1.0   
...             ...   ...       ...       ...       ...       ...        ...   
8522970  2024-03-08  SMGR   5900.00   5950.00   5900.00   5950.00  6695300.0   
8523047  2024-03-08  TKIM   6850.00   6950.00   6800.00   6850.00  1895200.0   
8523082  2024-03-08  TSPC   1940.00   1960.00   1940.00   1955.00   839100.0   
8523097  2024-03-08  UNSP    110.00    112.00    105.00    110.00    83600.0   
8523099  2024-03-08  UNTR  24175.00  24325.00  24025.00  24025.00  2820700.0   

          day  
13       

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:01<00:00, 20.98it/s]



boll_ub


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:01<00:00, 20.94it/s]



boll_lb


...


close_30_sma


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:01<00:00, 21.70it/s]



close_60_sma


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:01<00:00, 21.77it/s]


Successfully added technical indicators
Successfully added turbulence index


In [26]:
fprocessed = "/home/devmiftahul/trading_model/from_finrl-tutorials_git/processed_by_gavin/preprocessed_data_with_features.csv"
# processed.to_csv(fprocessed, index=False)

In [31]:
processed = pd.read_csv(fprocessed)
processed[20000:]

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
20000,1995-09-19,MLPL,804.00,804.00,788.00,788.00,1146764.0,667,-15.170880,867.380780,747.019220,47.792635,-53.000779,5.090404,823.200000,856.333333,10.128211
20001,1995-09-19,MPPA,896.00,896.00,885.00,896.00,489084.0,667,10.073460,976.224550,847.975450,53.334068,-18.159434,7.396098,902.766667,864.766667,10.128211
20002,1995-09-19,MYOR,56.53,56.53,55.67,55.67,2685425.0,667,-0.792743,59.966540,54.837460,47.210134,-69.980303,25.762406,58.686333,59.353167,10.128211
20003,1995-09-19,PNBN,201.00,201.00,201.00,201.00,5887918.0,667,-4.295826,222.425256,193.974744,36.402861,-115.302186,27.252667,210.000000,213.550000,10.128211
20004,1995-09-19,SMCB,692.00,696.00,692.00,692.00,2988988.0,667,10.758618,716.685990,620.114010,56.239604,112.869427,34.324170,659.833333,667.383333,10.128211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228295,2024-03-08,SMGR,5900.00,5950.00,5900.00,5950.00,6695300.0,7610,-77.012660,6470.763074,5799.236926,41.660961,-145.740281,26.024825,6160.000000,6194.166667,8.296629
228296,2024-03-08,TKIM,6850.00,6950.00,6800.00,6850.00,1895200.0,7610,76.026807,7065.827666,5951.672334,51.211188,137.327368,27.307389,6465.000000,6712.500000,8.296629
228297,2024-03-08,TSPC,1940.00,1960.00,1940.00,1955.00,839100.0,7610,-17.276449,2056.822728,1925.177272,48.258022,-117.575758,10.393875,2014.666667,1964.333333,8.296629
228298,2024-03-08,UNSP,110.00,112.00,105.00,110.00,83600.0,7610,0.606521,115.289621,104.410379,49.705775,11.675265,32.719668,108.366667,110.300000,8.296629
